# Building a Fraud Prediction Model with EvalML

In this demo, we will build an optimized fraud prediction model using EvalML. To optimize the model we will set up an objective function based on some assumptions about our business. 

In [1]:
%load_ext autoreload
%autoreload 2
import evalml
from evalml.objectives import FraudDetection
from evalml.preprocessing import split_data

/Users/kanter/.pyenv/versions/3.6.8/envs/evalml/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Load Transactions

In [2]:
X, y = evalml.demos.load_fraud()

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 99992

Labels
False    84.82%
True     15.18%
Name: fraud, dtype: object


## Define Objective

In [3]:
fraud_objective = FraudDetection(
    retry_percentage=.5,
    interchange_fee=.02,
    fraud_payout_percentage=.75,
    amount_col='amount'
)

## Search for best modeling pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [4]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']
X = X.select_dtypes(include=numerics)
X_train, X_holdout, y_train, y_holdout = split_data(X, y, test_size=.8, random_state=0)

Because the fraud labels are binary, we will use `AutoClassifier`. When we call `.fit()`, the search for the best pipeline will begin. 

In [5]:
clf = evalml.AutoClassifier(objective=fraud_objective,
                            max_time=120,
                            max_pipelines=20)

clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Fraud Detection. Lower score is better.

Searching up to 20 pipelines. Will stop searching for new pipelines after 60 seconds.

Possible model types: random_forest, xgboost, linear_model

Testing Random Forest w/ imputation:  55%|█████▌    | 11/20 [01:08<00:58,  6.48s/it]              

Max time elapsed. Stopping search early.
Testing Random Forest w/ imputation:  55%|█████▌    | 11/20 [01:08<00:55,  6.22s/it]

✔ Optimization finished


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the objective function we defined

In [6]:
clf.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,7,LogisticRegressionPipeline,2578148.10,False,"{'penalty': 'l2', 'C': 8.123565600467177, 'imp..."
1,6,LogisticRegressionPipeline,2585191.38,False,"{'penalty': 'l2', 'C': 0.5765626434012575, 'im..."
2,5,LogisticRegressionPipeline,2660723.71,False,"{'penalty': 'l2', 'C': 6.239401330891865, 'imp..."
3,8,LogisticRegressionPipeline,2667180.28,False,"{'penalty': 'l2', 'C': 8.362426847738403, 'imp..."
4,2,LogisticRegressionPipeline,2709059.45,False,"{'penalty': 'l2', 'C': 8.444214828324364, 'imp..."
5,4,RFPipeline,10464195.25,False,"{'n_estimators': 569, 'max_depth': 630, 'imput..."
6,0,XGBoostPipeline,11610654.75,False,"{'eta': 0.5928446182250184, 'min_child_weight'..."
7,1,XGBoostPipeline,11610654.75,False,"{'eta': 0.38438170729269994, 'min_child_weight..."
8,9,XGBoostPipeline,11610654.75,False,"{'eta': 0.6481718720511973, 'min_child_weight'..."
9,10,RFPipeline,11610654.75,False,"{'n_estimators': 369, 'max_depth': 10, 'impute..."


to select the best pipeline we can run

In [ ]:
pipeline = clf.best_pipeline

to select another pipeline we can use the id

In [ ]:
pipeline = clf.get_pipeline(0)

### Describe pipeline

You can get more details about any pipeline. Including 

In [ ]:
clf.describe_pipeline(0)